# Estruturas criptográficas 2024-2025
**Grupo 02**

**Pg55986:** Miguel Ângelo Martins Guimarães 

**Pg55997:** Pedro Miguel Oliveira Carvalho 

## Setup

### Imports

In [905]:
# Imports
from sage.all import *
import secrets
from cryptography.hazmat.primitives import hashes

In [906]:
# Parametros para a curva edwards22519
p = 2^255-19
K = GF(p)
a = K(-1)
d = -K(121665)/K(121666)

ed25519 = {
    'b'  : 256,
    'Px' : K(15112221349535400772501151409588531511454012693041857206046113283949847762202),
    'Py' : K(46316835694926478169428394003475163141307993866256225615783033603165251855960),
    'L'  : ZZ(2^252 + 27742317777372353535851937790883648493), ## ordem do subgrupo primo
    'n'  : 254,
    'h'  : 8
}

############ Verificar os domain parameteres

### Parametros do codigo

In [907]:
##########################
# Parametros / setup
debug = True
##########################

## Definição de classes

### Classe da curva de edwards

In [908]:
# Classe de criação de curva de edwards/weiterstrass. 
# É necessario utilizar as curvas de weiterstrass porque o sagemath já as implementa, entao podemos utilizar para definir curvas de edwards assim.
class Ed(object):
    # Construtor da classe que prepara alguns parametros auxiliares
    def __init__(self, p, a, d, ed = None):
        # Garantir que 'a' e 'd' são diferentes, e que 'p' é um numero primo grande
        assert a != d and is_prime(p) and p > 3
        
        # Criar um corpo finito modulo p (permite realizar varias operações)
        K = GF(p) 
  
        # Valores que permitem mapear a curva de edwards para weiterstrass
        A =  2*(a + d)/(a - d)
        B =  4/(a - d)
        alfa = A/(3*B)  
        s = B

        # Coeficientes na curva de weiterstrass (y^2 = x^3 + a4 * x + a6)
        a4 = s^(-2) - 3*alfa^2
        a6 = -alfa^3 - a4*alfa
        
        # Parametros para a criação da curva 
        self.K = K # Guardar o corpo finito
        self.constants = {'a': a , 'd': d , 'A':A , 'B':B , 'alfa':alfa , 's':s , 'a4':a4 , 'a6':a6 }
        self.EC = EllipticCurve(K,[a4,a6]) # Criação da curva eliptica na forma de weiterstrass 
        
        # Se forem fornecidos parametros como input
        if ed != None:
            self.L = ed['L'] # Obter o L apartir do dicionario
            self.P = self.ed2ec(ed['Px'],ed['Py']) # Criar um gerador de pontos de weiterstrass apartir dos pontos de edwards fornecidos
        else:
            self.gen() # Gera aleatoriamente um ponto na curva que sirva de gerador
    
    # Devolve a ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h" 
    def order(self):
        oo = self.EC.order() # Numero total de pontos na curva
        n,_ = list(factor(oo))[-1] # Obter fatores
        return (n,oo//n) # Retorna o maior subgrupo primo e o cofator
    
    # Gera aleatoriamente um ponto na curva que sirva de gerador
    def gen(self):
        L, h = self.order()       
        P = O = self.EC(0)
        while L*P == O:
            P = self.EC.random_element()
        self.P = h*P ; self.L = L
  
    # Função que verifica se um par de pontos pertence á curva de edwards
    def is_edwards(self, x, y):
        a = self.constants['a'] # Obter o valor de a declarado (parametro da curva)
        d = self.constants['d'] # Obter o valor de d declarado (parametro da curva)
        x2 = x^2 # Calcular o valor de x ao quadrado  
        y2 = y^2 # Calculcar o valor de y ao quadrado
        return a*x2 + y2 == 1 + d*x2*y2 # Equação da curva de twisted edwards curves (a * x^2 + y^2 = 1 + d * x^2 * y^2)

    # Passa um ponto de curva de edwards para um ponto de curva de ec
    def ed2ec(self,x,y):      
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) 
        w = z/x
        alfa = self.constants['alfa']
        s = self.constants['s']
        return self.EC(z/s + alfa , w/s)
    
    # Passa um ponto de curva de ec para curva de edwards
    def ec2ed(self,P):    
        if P == self.EC(0):
            return (0,1)
        x,y = P.xy() # Obter o valor x e y
        alfa = self.constants['alfa']
        s = self.constants['s']
        u = s*(x - alfa)  
        v = s*y
        return (u/v , (u-1)/(u+1))

### Classe dos pontos das curvas de edwards

In [909]:
# Points class of ED
class ed(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x ; self.y = pt.y ; self.w = pt.w
        else:
            assert isinstance(curve,Ed) and curve.is_edwards(x,y)
            self.curve = curve
            self.x = x ; self.y = y ; self.w = x*y
    
    # Verifica se dois pontos são equivalentes
    def eq(self,other):
        return self.x == other.x and self.y == other.y
    
    # Devolve uma copia do ponto atual
    def copy(self):
        return ed(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return ed(curve=self.curve,x=0,y=1)
    
    def sim(self):
        return ed(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*self.w*other.w
        self.x, self.y  = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        
    def duplica(self):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*(self.w)^2
        self.x, self.y = (2*self.w)/(1+delta) , (self.y^2 - a*self.x^2)/(1 - delta)
        self.w = self.x*self.y
        
    def mult(self, n):
        m = Mod(n,self.curve.L).lift().digits(2)   ## obter a representação binária do argumento "n"
        Q = self.copy() ; A = self.zero()
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        return A

### Encoder de pontos

In [910]:
# Encoding para representar um ponto para bytes
def encode_points(x, y):
    # Verificar se os pontos pertecem a uma curva de edwards
    E = Ed(p, a, d, ed25519) # Criar instancia do edwards
    if not E.is_edwards(x, y):
        raise ValueError("O ponto (x, y) não pertence à curva Edwards definida.")

    # Passar para ints
    x_int = int(x) # Passar o valor x para int
    y_int = int(y) # Passar o valor y para int

    # Passo 1: "Encode the y-coordinate as a little-endian string of 32 octets"
    y_bytes = y_int.to_bytes(32, byteorder='little')

    # Passo 2: "Copy the least significant bit of the x-coordinate to the most significant bit of the final octet"
    sign_bit = x_int & 1  # Extrair o ultimo bit do x

    # Copiar o LSB de x para o MSB do último octeto de y
    y_byteArray = bytearray(y_bytes)
        
    if sign_bit == 1:
        y_byteArray[31] |= 0x80  # Define o MSB para 1 (0x80 = 10000000 em binário)
    else:
        y_byteArray[31] &= 0x7F  # Garante que o MSB seja 0 (0x7F = 01111111 em binário)

    result = bytes(y_byteArray)

    # Modo debug
    if(debug):
        print("[ENCODE] ------- INPUT ------- ")
        print("[ENCODE] Recebido x: " + str(x))
        print("[ENCODE] Recebido y: " + str(y))
        print()

    return result

### Decoder de pontos

In [911]:
# Função auxiliar do decoder de pontos que permite obter a raiz modulo (x^2 = n mod p)
# Irei seguir o (NIST SP 800-186, Appendix E).
def tonelliShanks(x2, p):
    n = x2

    # 1 passo: "Find q and s (with q odd), such that p–1 = q * 2^s"
    # Vamos iterar o q e o s até que nao seja possivel simplificar mais a equação 
    Q = p - 1 # Inicializar o q com valor igual a p-1, ou seja se o s for 0
    S = 0    

    # 2 passo: Aplicar verificar se o Q ainda é divisivel por 2, se for então é possivel aumentar o s em 1 denovo
    while Q % 2 == 0:
        Q = Q // 2 # Dividir o q por 1 pois o valor de 2 vai passar a ser representado por um incremento no S (// é divisão inteira)
        S += 1 

    # 3 passo: "Check to see if nq = 1"
    t = pow(n, Q, p)  # n^q mod p
    if t == 1:
        # "If so, then the root x = n(q+1)/2 mod p"
        return pow(n, (Q+1)//2, p) # Solução imediata se existir
    
    # Prosseguir a escolher z e entrando no loop principal
    else:
        # 3 passo: "Select a z, which is a quadratic non-residue modulo p"
        # Nota: "The Legendre symbol (a/p) where p is an odd prime and prime and a is an integer, can be used to test candidate values for z to see if a value of –1 is returned"
        found_candidate = 0
        for candidate in range(2, p):
            # Usando a identidade de euler (z/p) == z^(p-1/2)
            ls = pow(candidate, (p - 1) // 2, p)
            if ls == p - 1:  # pois p-1 ≡ -1 (mod p)
                found_candidate = candidate
                break
        
        # Se nao existir candidato para Z matar tudo
        if found_candidate == 0:
            raise ValueError("[TONELLI-SHANKS] Não foi encontrado QNR")
        
        # Passo 5: Inicialização dos valores para o loop
        c = pow(z, Q, p) # "Set c = zq mod p. " 
        t = pow(n, Q, p) # "Set t = nq mod p.""
        R = pow(n, (Q + 1)//2, p) # "Set x = n(q+1)/2 mod p."
        M = S # "Set m = s"

        # Passo 6: "While t != 1"
        while t != 1:
            # Passo 7: "Find the smallest i, such that 𝑡^2^i = 1"
            t2i = t # Definir valor auxiliar incial

            # Iterar todos os valores de i até que t^2^i = 1
            i = 0
            for i in range(1, M): # Esta definido que "0 < i < m"
                t2i = pow(t2i, 2, p)
                if t2i == 1:
                    break # Foi encontrado um i que torna isto valido
            
            # Situação rara de erro
            if i == M:
                raise ValueError("[TONELLI-SHANKS] Não foi encontrado um valor valido de i")
            
        # Atualizar os valores
        b = pow(c, 1 << (M - i - 1), p) # 1 << k computa 2^k, com este shift estamos a simplificar a formula
        x = (R * b) % p

        return x


In [912]:
# Faz decode de uma string de bytes para um ponto y e x (encoded: pontos_da_curva, d:coeficiente_curva, p:numero_primo_corpo)
def decode_points(encoded, d, p):
    ####################################### PRIMEIRA PARTE (Obter o valor x0 e o valor y) ####################################### 
    # 1 Passo: "Interpret the octet string as an integer in little-endian representation"
    enc_val = int.from_bytes(encoded, byteorder="little")

    # 2 Passo: "The most significant bit of this integer is the least significant bit of the x-coordinate"
    x0 = (enc_val >> 255) & 1 # Damos shift do bit que queremos para a ultima posição e extraimos com o &1

    # 3 Passo: "The ycoordinate is recovered simply by clearing this bit"
    y_val = enc_val & ((1 << 255) - 1) # Criar sequencia de bits "valor & (...)111111110" que irá colocar o msb a 0

    # 4 Passo: "If the resulting value is ≥ p, decoding fails"
    if y_val >= p:
        raise ValueError("Decodificação falhou: y_val >= p.")

    ####################################### SEGUNDA PARTE (Obter o valor x2) #######################################
    # Ao pegar na equação da curva de edwards é possivel reorganiza-la para ficar ordem x^2
    # x^2 = (y^2 - 1) / (d y^2 - a) (mod p). O objetivo será calcular o x^2. 

    # 1 Passo: Calcular y^2 (Mod p porque toda a aritmetica é feita mod p em corpos finitos)
    y2 = pow(y_val, 2, p)

    # 2 Passo: Calcular o numerador e denominador
    N = (y2 - 1) % p      # Numerador = y^2 - 1
    D = (d * y2 - 1) % p  # D = d * y^2 - 1

    # 3 Passo: Verificar se o denominador dá 0
    if D == 0:
        raise ValueError("Decodificação falhou: denominador zero na equação")
    
    # Esta equação simplifica a primeira equação: x^2 ≡ N * D^−1 (mod p)
    # Vamos tentar obter o x2 utilizando o N e o D. Não é possivel dividir um numero por outro em aritmetica modular.
    # Entao teremos de utilizar multiplicação

    # Passo 3: Obter o inverso de D (Aplicando teorema de Fermat)
    D_inv = pow(D, p-2, p)

    # Passo 4: Aplicar a equação para obter o valor de x^2
    x2 = (N * D_inv) % p

    # Passo 5: Obter o x
    x = tonelliShanks(x2,p)

    ####################################### TERCEIRA PARTE (Obter o valor de x) #######################################
    # Para obter o valor de x2 teremos de utilizar a formula de Tonelli-Shanks, que permite obter raiz quadrada modulo 
    # Irei seguir o (NIST SP 800-186, Appendix E).

    # Modo debug
    if(debug):
        print("[DECODE] ------- INPUT ------- ")
        print("[DECODE] Recebido d: " + str(d))
        print("[DECODE] Recebido p: " + str(p))
        print("[DECODE] ------- FASE 1 ------- ")
        print("[DECODE] Obtido y: " + str(y_val))
        print("[DECODE] Obtido x0: " + str(x0))
        print("[DECODE] ------- FASE 2 ------- ")
        print("[DECODE] Obtido y2: " + str(y2))
        print("[DECODE] Obtido x2: " + str(x2))
        print("[DECODE] Obtido x: " + str(x))


if(debug):
    print("TESTE DA FUNÇÃO")
    val = encode_points(15112221349535400772501151409588531511454012693041857206046113283949847762202, 46316835694926478169428394003475163141307993866256225615783033603165251855960)
    d = -K(121665)/K(121666)
    decode_points(val, d, 2^255-19)

TESTE DA FUNÇÃO
[ENCODE] ------- INPUT ------- 
[ENCODE] Recebido x: 15112221349535400772501151409588531511454012693041857206046113283949847762202
[ENCODE] Recebido y: 46316835694926478169428394003475163141307993866256225615783033603165251855960

[DECODE] ------- INPUT ------- 
[DECODE] Recebido d: 37095705934669439343138083508754565189542113879843219016388785533085940283555
[DECODE] Recebido p: 57896044618658097711785492504343953926634992332820282019728792003956564819949
[DECODE] ------- FASE 1 ------- 
[DECODE] Obtido y: 46316835694926478169428394003475163141307993866256225615783033603165251855960
[DECODE] Obtido x0: 0
[DECODE] ------- FASE 2 ------- 
[DECODE] Obtido y2: 37053468555941182535542715202780130513046395093004980492626426882532201484768
[DECODE] Obtido x2: 54282905433475203821040348471106652382200921153486091201946808333483576607989
[DECODE] Obtido x: 6911807055129475468602356771431595722268599919236466436918482538819257280048


## Implementação do ed25519

### Gerar par de chaves

In [913]:
# Função que gera um par de chaves 
def genKeys(G):
    # Criação do par de chaves (Parametros)
    b = 256 # Tamanho Especifico ao ed25519         
    requested_security_strenght = 32 # Tamanho Especifico ao ed25519 em bytes (32 bytes = 128 bits)  
    keysDigest = hashes.Hash(hashes.SHA512()) # Utilização do sha512 como hash (Especificação do ed25519)

    # Passo 1 (Gerar chave privada)
    private_key = secrets.token_bytes(requested_security_strenght)  

    # Passo 2 (Calcular a hash da chave privada) 
    keysDigest.update(private_key)
    privateKey_hash = keysDigest.finalize()

    # Passo 3 (Gerar a chave publica)
    metade = privateKey_hash[:len(privateKey_hash)//2] # Obter a primeira metade do hash gerado

    # Passo 3.1 ("The first 32 octets of H are interpreted as a little-endian integer...")
    metade_byteArray = bytearray(metade)
    metade_byteArray[0] &= 248       # Limpar bits 0,1,2 ("and 11111000" que faz com que os 3 primeiros bits se tornem 0) (O algoritmo utiliza notação little endian)
    metade_byteArray[31] &= 127      # Limpar bit 7 ("and 01111111" que faz com que o ultimo bit se torne 0) (O algoritmo utiliza notação little endian)
    metade_byteArray[31] |= 64       # Definir bit 6 a 0 ("or 01000000" que faz com que o penuultimo bit se torne 1 e mantem os restantes) (O algoritmo utiliza notação little endian)
    metade_novo = bytes(metade_byteArray)

    # Passo 4 ("Determine an integer s from hdigest1 using little-endian convention")
    s = int.from_bytes(metade_novo, byteorder="little")

    # Passo 5 ("Compute the point [s]G. The corresponding EdDSA public key Q is the encoding of the point [s]G")
    # Criar objeto da curva
    pub_point = G.mult(s) # Obter [s]G
    public_key = encode_points(pub_point.x, pub_point.y) # "The corresponding EdDSA public key Q is the encoding of the point [s]G" 

    if(debug):
        print("Key pair generated!")
        print("Public key (hex):", public_key.hex())
        print("Private key (hex):", private_key.hex())

    return private_key, public_key

### Gerar assinatura EdDSA

In [914]:
# Input: chave publica, chave privada, gerador g na curva de edwards (tem de ser o mesmo que nas chaves)
def genSignature(Message, public_key, private_key, G, n):
    # Passo 1 (Computar a hash da chave privada) 
    digest1 = hashes.Hash(hashes.SHA512()) # Utilização do sha512 como hash (Especificação do ed25519)
    digest1.update(private_key)
    privateKeyHash = digest1.finalize()

    # Passo 2: Extrair a segunda parte do hash (últimos 32 bytes)
    privateKeyHash_half = privateKeyHash[len(privateKeyHash)//2:] # Pegar em todos os bytes desde metade do comprimento

    # Passo 2.1: r = SHA-512(hdigest2 || M)
    digest2 = hashes.Hash(hashes.SHA512()) # Utilização do sha512 como hash (Especificação do ed25519)
    conc = privateKeyHash_half + Message
    digest2.update(conc)
    r = digest2.finalize()

    # Passo 2.2 (NÃO APLICAVEL, Utilizado pelo ed448)
    # Passo 3 (Compute the point [r]G. The octet string R is the encoding of the point [r]G)
    r_int = int.from_bytes(r, byteorder="little") 
    R_point = G.mult(r_int)
    R_encoded = encode_points(R_point.x, R_point.y)
    r_encoded_int = int.from_bytes(R_encoded, byteorder="little") 

    # Passo 4: "Derive s from H(d) as in the key pair generation algorithm"
    digest4 = hashes.Hash(hashes.SHA512())
    digest4.update(private_key)
    privateHash = digest4.finalize()

    # Roubado da etapa de gerar chaves tal como pedido
    metade_byteArray = bytearray(privateHash)
    metade_byteArray[0] &= 248       # Limpar bits 0,1,2 ("and 11111000" que faz com que os 3 primeiros bits se tornem 0) (O algoritmo utiliza notação little endian)
    metade_byteArray[31] &= 127      # Limpar bit 7 ("and 01111111" que faz com que o ultimo bit se torne 0) (O algoritmo utiliza notação little endian)
    metade_byteArray[31] |= 64       # Definir bit 6 a 0 ("or 01000000" que faz com que o penuultimo bit se torne 1 e mantem os restantes) (O algoritmo utiliza notação little endian)
    metade_novo = bytes(metade_byteArray)

    s = int.from_bytes(metade_novo, byteorder="little")

    # Passo 4.1: Calcular S = (r + SHA-512(R || Q || M) * s) mod n
    conc_s = R_encoded + public_key + Message # R || Q || M
    digest3 = hashes.Hash(hashes.SHA512())  # Utilização do sha512 como hash (Especificação do ed25519)
    digest3.update(conc_s)
    conc_hash = digest3.finalize() # SHA-512(R || Q || M)

    # Converter o digest para inteiro usando little-endian
    h_int = int.from_bytes(conc_hash, byteorder="little")

    # Reduzir modulo n
    h_int_mod = h_int % n

    # Calcular S = (r_int + h_int_mod * s_scalar) mod n
    S_int = (r_int + h_int_mod * s) % n

    # Converter S_int para 32 bytes (little-endian)
    S_bytes = S_int.to_bytes(32, byteorder="little")

    # "The octet string S is the encoding of the resultant integer"
    S_final = int.from_bytes(S_bytes, byteorder="little")

    # Passo 5: Form the signature as the concatenation of the octet strings R and S. 
    result = r_encoded_int + S_final
    return result

### Verificação de assinaturas

In [915]:
# Cria um ponto gerador
E = Ed(p, a, d, ed25519)    # Criar instancia do edwards
gx, gy = E.ec2ed(E.P)       # Obter um ponto na curva de edwards (x,y) atraves de um ponto na curva de weiterstrass "P" (ponto gerado automaticamente)
G = ed(curve=E, x=gx, y=gy) # Representa o gerador de uma curva de edwards
n = ZZ(2^252 + 27742317777372353535851937790883648493) ## ordem do subgrupo primo

private_key, public_key = genKeys(G)
sig = genSignature(b"ola", public_key, private_key, G, int(n))
print("Assinatura:" + str(sig))

[ENCODE] ------- INPUT ------- 
[ENCODE] Recebido x: 8300226965333892783186792966933989523772687605347623445708314890219025133650
[ENCODE] Recebido y: 35656263976627119384980098341399600796663092239533711445044632196025334334505

Key pair generated!
Public key (hex): 2984c72e471f121b69b1b868539758630934557171eef2f216ebc186f0bad44e
Private key (hex): 483cefd3338c6f48bcecf328cbfe40412248fac41be02a0f815893990f9b91b3
[ENCODE] ------- INPUT ------- 
[ENCODE] Recebido x: 16562348373475711412018606154948488139263591468313994811166297380315966905850
[ENCODE] Recebido y: 26861574058784714206043786935143762400370669627428491273549210441912226389547

Assinatura:32455062088060697063141129850907207732365730025769137015258213770290282903270
